## Step 1: Saving the files in pkl format

iwspa train and test is just copied and pasted. SpamSMS is processed

In [4]:
import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv("/disk2/sadat/FakeNewsData/archive/spam.csv", encoding = 'latin-1')
df.rename(columns={"v1":"labels", "v2":"text"}, inplace=True)
spam = df[["labels", "text"]]
#spam = spam.sample(frac=1, random_state=100).reset_index(drop=True)
spam.head()

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
spam["is_phishing"] = spam["labels"].apply(lambda x:1 if x=="spam" else 0)
spam.to_pickle("/disk2/sadat/PhishingResearch/processed_work/psych_trait_experiments/Files/spamSMS.pkl")
spam.head()

<ipython-input-8-1165b68f4273>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spam["is_phishing"] = spam["labels"].apply(lambda x:1 if x=="spam" else 0)


,labels,text,is_phishing
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


## Step 2: Tokenization

In [10]:
import time
import pandas as pd 
import numpy as np
import pickle
import time
import os
import sys
import logging
from transformers import BertTokenizer


def tokenization_all_datasets(path, one_file_only=False):
    start = time.time()
    if one_file_only==False:
        all_files = os.listdir(path)
        for files in all_files:
            df = pd.read_pickle(path + files)
            print(files)
            tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
            df["tokenized"] = df["text"].apply(lambda sent:tokenizer.encode(sent, add_special_tokens=True, 
                                                                                  max_length=512, truncation=True,
                                                                                  padding='max_length', 
                                                                                  return_attention_mask=False))
            df.to_pickle(path + files)
            end = time.time()
            elapsed = end-start
            x = files + "is_done in " + str("%.2f" %(elapsed)) + " seconds\n"
    else:
        df = pd.read_pickle(path)
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        df["tokenized"] = df["text"].apply(lambda sent:tokenizer.encode(sent, add_special_tokens=True, 
                                                                              max_length=512, truncation=True,
                                                                              padding='max_length', 
                                                                              return_attention_mask=False))
        df.to_pickle(path)
        end = time.time()
        elapsed = end-start        

tokenization_all_datasets("/disk2/sadat/PhishingResearch/processed_work/psych_trait_experiments/Files/spamSMS.pkl", 
                          one_file_only=True)


In [11]:
df = pd.read_pickle("/disk2/sadat/PhishingResearch/processed_work/psych_trait_experiments/Files/spamSMS.pkl")

In [12]:
df.head()

,labels,text,is_phishing,tokenized
0,ham,"Go until jurong point, crazy.. Available only ...",0,"[101, 2175, 2127, 18414, 17583, 2391, 1010, 46..."
1,ham,Ok lar... Joking wif u oni...,0,"[101, 7929, 2474, 2099, 1012, 1012, 1012, 1664..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,"[101, 2489, 4443, 1999, 1016, 1037, 1059, 2243..."
3,ham,U dun say so early hor... U c already then say...,0,"[101, 1057, 24654, 2360, 2061, 2220, 7570, 209..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"[101, 20976, 1045, 2123, 1005, 1056, 2228, 200..."


## Step 3: Saving BERT 

In [44]:
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
import numpy as np
import os

In [63]:
def create_data_set(datapath):
    df = pd.read_pickle(datapath)
    train_text = np.array(list(df["tokenized"]))
    attention_masks = np.where(train_text>0, 1, 0)
    return torch.tensor(train_text), torch.tensor(attention_masks)

def create_dataloader(train_text, attention_masks, batch_size=512):
    # Create the DataLoader for our training set
    '''
    This function will create a dataloader for our training set. The dataloader will help to feed the randomly 
    sampled data on each batch. The batch size is selected to be 16, is simply as instructed in the original
    paper. 
    '''
    train_data = TensorDataset(train_text, attention_masks)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return train_dataloader

def BERT_features_saved(datapath):
    for filename in [f for f in os.listdir(datapath) if f.endswith(".pkl")]:
        input_ids, att_mask = create_data_set(datapath+filename)
        device = torch.device("cuda")
        bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)
        loader = create_dataloader(input_ids, att_mask, batch_size=16)
        All_batch = []
        for step, batch in enumerate(loader):
            token, mask = batch
            outputs = bert_model(token.to(device), mask.to(device))
            last_hidden_state_cls = outputs[0][:, 0, :]
            np_array = last_hidden_state_cls.detach().cpu().numpy()
            All_batch.append(np_array)
            if step%100==0:
                print("{} completed {} percent".format(filename, step*100/len(loader)))
                      
        p = np.concatenate(All_batch, axis=0)
        npy_file_path = datapath + "BERT_" + filename.split(".pkl")[0] + ".npy"
        saved_path = np.save(npy_file_path, p)
    

In [46]:
BERT_features_saved("/disk2/sadat/PhishingResearch/processed_work/psych_trait_experiments/Files/")

spamSMS.pkl completed 0.0 percent
spamSMS.pkl completed 28.653295128939828 percent
spamSMS.pkl completed 57.306590257879655 percent
spamSMS.pkl completed 85.95988538681948 percent
combined_iwspa_ap_train_set.pkl completed 0.0 percent
combined_iwspa_ap_train_set.pkl completed 27.932960893854748 percent
combined_iwspa_ap_train_set.pkl completed 55.865921787709496 percent
combined_iwspa_ap_train_set.pkl completed 83.79888268156425 percent
iwspa_ap_test.pkl completed 0.0 percent
iwspa_ap_test.pkl completed 37.174721189591075 percent
iwspa_ap_test.pkl completed 74.34944237918215 percent


In [53]:
from sentence_transformers import SentenceTransformer

#Sentences are encoded by calling model.encode()


You try to use a model that was created with version 1.2.0, however, your version is 1.0.3. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





CPU times: user 7.3 s, sys: 504 ms, total: 7.81 s
Wall time: 2.25 s


In [ ]:
model = SentenceTransformer('paraphrase-mpnet-base-v2').to(device)

In [57]:
sentence_embeddings = model.encode(spam["text"])

In [72]:
def SBERT_features_saved(datapath):
    for filename in [f for f in os.listdir(datapath) if f.endswith(".pkl")]:
        df = pd.read_pickle(datapath + filename)
        device = torch.device("cuda")
        sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2').to(device)
        sentence_embeddings = sbert_model.encode(df["text"])
        npy_file_path = datapath + "SBERT_" + filename.split(".pkl")[0] + ".npy"
        saved_path = np.save(npy_file_path, p)

In [73]:
SBERT_features_saved("/disk2/sadat/PhishingResearch/processed_work/psych_trait_experiments/Files/")

You try to use a model that was created with version 1.2.0, however, your version is 1.0.3. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



You try to use a model that was created with version 1.2.0, however, your version is 1.0.3. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



You try to use a model that was created with version 1.2.0, however, your version is 1.0.3. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



